An example yaml config

```yaml
model:
  model_name: batch_reactor_test
  model_type: continuous
  constants:
    mu_m: 0.02
    K_m: 0.05
    K_i: 5.0
    v_par: 0.004
    Y_p: 1.2

  state_variables:
    X_s:
      init_val: 1.0
      init_val_lower: 0.9
      init_val_upper: 1.1
      rhs: mu_S*X_s - inp/V_s*X_s
      shape: (1,) # default value, can be omitted
      scaling: 1 # default value, can be omitted
    S_s:
      init_val: 0.5
      init_val_lower: 0.4
      init_val_upper: 0.6
      rhs: mu_S*X_s/Y_x - v_par*X_s/Y_p + inp/V_s*(S_in-S_s)
    P_s:
      init_val: 0.0
      init_val_lower: 0.0
      init_val_upper: 0.2
      rhs: v_par*X_s - inp/V_s*P_s
    V_s:
      init_val: 120.0
      init_val_lower: 110.0
      init_val_upper: 130.0
      rhs: inp

  control_variables:
    inp:
      shape: (1,) # default value, can be omitted

  user_defined_parameters:
    - Y_x
    - S_in
  aux_variables:
    mu_S:
      expr: mu_m*S_s/(K_m+S_s+(S_s**2/K_i))
```
---- 

# Define your model

In [3]:
from vis_utility import *

header  = Button(description='Model Specification',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
model_name = Text(value='Model Name',
                  placeholder='Type something',
                  description='Model Name:',
                  disabled=False)

model_type = Dropdown(options=['continuous', 'discrete'],
                      value='continuous',
                      description='Model Type:',
                      disabled=False)


num_constants = BoundedIntText(value=0,
                               min=0,
                               step=1,
                               description='Num. Constants:',
                               disabled=False)


num_state_variables = BoundedIntText(value=0,
                                     min=0,
                                     step=1,
                                     description='Num. State Variables:',
                                     disabled=False)



num_control_variables = BoundedIntText(value=0,
                                       min=0,
                                       step=1,
                                       description='Num. Control Variables:',
                                       disabled=False)


num_parameters = BoundedIntText(value=0,
                                min=0,
                                step=1,
                                description='Num. User Defined Parameters',
                                disabled=False)


num_aux_variables = BoundedIntText(value=0,
                                   min=0,
                                   step=1,
                                   description='Num. Auxiliary Variables:',
                                   disabled=False)


children=[model_name, model_type, num_constants,
          num_state_variables, num_control_variables,
          num_parameters, num_aux_variables]

grid = layout_generator(header, children, col_width=400)
display(grid)


GridspecLayout(children=(Button(description='Model Specification', layout=Layout(grid_area='widget001', width=…

In [6]:
# define details of each variable/constant/parameter

if num_constants.value > 0:
    constant_children = []
    constant_names = []
    constant_values = []
    constants_header = Button(description='Constants Definition',
                     layout=Layout(width='auto', grid_area='header'),
                     style=ButtonStyle(button_color='lightblue'))
    for i in range(num_constants.value):
        constant_def = Text(value=f"X_{i}",
                            placeholder=f"X_{i}",
                            description=f"{i+1}-th Constant Name:",
                            disabled=False)

        constant_val = FloatText(value=0.0,
                                 description=f"Value:",
                                 disabled=False)
        constant_children.extend([constant_def, constant_val])
        constant_names.append(constant_def)
        constant_values.append(constant_val)


    grid = layout_generator(constants_header, constant_children, col_width=400)
    display(grid)


GridspecLayout(children=(Button(description='Constants Definition', layout=Layout(grid_area='widget001', width…

In [27]:
"""
state_variables:
    X_s:
      init_val: 1.0
      init_val_lower: 0.9
      init_val_upper: 1.1
      rhs: mu_S*X_s - inp/V_s*X_s
      shape: (1,) # default value, can be omitted
      scaling: 1 # default value, can be omitted
"""

if num_state_variables.value > 0:
    state_variables_children = []
    state_variables_names = []
    state_variables_init_values = []
    state_variables_lower_bounds = []
    state_variables_upper_bounds = []
    state_variables_rhs = []
    state_variables_shape = []
    state_variables_scaling = []
    state_variables_soft = []
    
    state_variables_header = Button(description='State Variables Definition',
                     layout=Layout(width='auto', grid_area='header'),
                     style=ButtonStyle(button_color='lightblue'))
    for i in range(num_constants.value):
        sv_def = Text(value=f"X_{i}",
                            placeholder=f"X_{i}",
                            description=f"{i+1}-th State Variable Name:",
                            disabled=False)

        sv_init_val = FloatText(value=0.0,
                                 description=f"Init Value:",
                                 disabled=False)
        
        sv_lower_bound = FloatText(value=0.0,
                                  description=f"Lower Bound:",
                                  disabled=False)
        
        sv_upper_bound = FloatText(value=0.0,
                                   description=f"Upper Bound:",
                                   disabled=False)
        
        sv_rhs = Text(value="x1+x2",
                            placeholder="x1+x2",
                            description="RHS:",
                            disabled=False)

        sv_shape = Text(value="(1,)",
                        placeholder="(1,)",
                        description="Shape:",
                        disabled=False)
                      
        sv_scaling = FloatText(value=1.0,
                               description=f"Scaling:",
                               disabled=False)
        
        sv_soft = Checkbox(value=False,
                           description='Soft',
                           disabled=False,
                           indent=False)

        
        state_variables_children.extend([sv_def, sv_init_val, sv_lower_bound, sv_upper_bound, sv_rhs, sv_shape, sv_scaling, sv_soft])
        state_variables_names.append(sv_def)
        state_variables_init_values.append(sv_init_val)
        state_variables_lower_bounds.append(sv_lower_bound)
        state_variables_upper_bounds.append(sv_upper_bound)
        state_variables_rhs.append(sv_rhs)
        state_variables_shape.append(sv_shape)
        state_variables_scaling.append(sv_scaling)
        state_variables_soft.append(sv_soft)

    grid = layout_generator(state_variables_header, state_variables_children, num_cols=8, col_width=200)
    display(grid)
    
    

GridspecLayout(children=(Button(description='State Variables Definition', layout=Layout(grid_area='widget001',…

In [30]:
"""
control_variables:
    inp:
      shape: (1,) # default value, can be omitted
"""

if num_control_variables.value > 0:
    control_variables_children = []
    control_variables_names = []
    control_variables_shapes = []
    control_variables_header = Button(description='Control Variables Definition',
                     layout=Layout(width='auto', grid_area='header'),
                     style=ButtonStyle(button_color='lightblue'))
    for i in range(num_control_variables.value):
        cv_def = Text(value=f"In_{i}",
                      placeholder=f"In_{i}",
                      description=f"{i+1}-th Control Variable:",
                      disabled=False)

        cv_shape = Text(value="(1,)",
                        placeholder="(1,)",
                        description="Shape:",
                        disabled=False)
        control_variables_children.extend([cv_def, cv_shape])
        control_variables_names.append(cv_def)
        control_variables_shapes.append(cv_shape)


    grid = layout_generator(control_variables_header, control_variables_children, col_width=400)
    display(grid)



GridspecLayout(children=(Button(description='Control Variables Definition', layout=Layout(grid_area='widget001…

In [19]:
"""
  user_defined_parameters:
    - Y_x
    - S_in
"""

if num_parameters.value > 0:
    parameters_children = []
    parameters_names = []
    parameters_uncertainities = []
    parameters_header = Button(description='User Defined Parameter Definition',
                     layout=Layout(width='auto', grid_area='header'),
                     style=ButtonStyle(button_color='lightblue'))
    for i in range(num_parameters.value):
        para_def = Text(value=f"Parameter_{i}",
                      placeholder=f"Parameter_{i}",
                      description=f"{i+1}-th Parameter:",
                      disabled=False)
        
        para_uncertainties = Text(value="1,2,3",
                                  placeholder="1,2,3",
                                  description="Values:",
                                  disabled=False)

        parameters_children.extend([para_def, para_uncertainties])
        parameters_uncertainities.append(para_uncertainties)
        parameters_names.append(para_def)


    grid = layout_generator(parameters_header, parameters_children, col_width=400)
    display(grid)


GridspecLayout(children=(Button(description='User Defined Parameter Definition', layout=Layout(grid_area='widg…

In [16]:
"""
aux_variables:
    mu_S:
      expr: mu_m*S_s/(K_m+S_s+(S_s**2/K_i))
"""

if num_aux_variables.value > 0:
    aux_variables_children = []
    aux_variables_names = []
    aux_variables_expr = []
    aux_variables_header = Button(description='Auxiliary Variables Definition',
                     layout=Layout(width='auto', grid_area='header'),
                     style=ButtonStyle(button_color='lightblue'))
    for i in range(num_aux_variables.value):
        av_def = Text(value=f"Aux_Variable_{i}",
                      placeholder=f"Aux_Variable_{i}",
                      description=f"{i+1}-th Aux. Variable:",
                      disabled=False)
        
        av_expr = Text(value=f"Expr_{i}",
                      placeholder=f"Expr_{i}",
                      description=f"Expr:",
                      disabled=False)

        aux_variables_names.append(av_def)
        aux_variables_expr.append(av_expr)
        aux_variables_children.extend([av_def, av_expr])


    grid = layout_generator(aux_variables_header, aux_variables_children, col_width=400)
    display(grid)



GridspecLayout(children=(Button(description='Auxiliary Variables Definition', layout=Layout(grid_area='widget0…

An example yaml config

```yaml

simulator:
  parameters:
    integration_tool: 'cvodes'
    abstol: 1.0e-10
    reltol: 1.0e-10
    t_step: 1.0
```
---- 

In [23]:
# define the simulator

num_simulator_parameters = BoundedIntText(value=0,
                                           min=0,
                                           step=1,
                                           description='Num. Simulator Parameters:',
                                           disabled=False)
num_simulator_parameters

BoundedIntText(value=0, description='Num. Simulator Parameters:')

In [25]:
if num_simulator_parameters.value > 0:
    num_simulator_parameters_children = []
    num_simulator_parameters_names = []
    num_simulator_parameters_values = []
    num_simulator_parameters_header = Button(description='Simulator Parameters',
                     layout=Layout(width='auto', grid_area='header'),
                     style=ButtonStyle(button_color='lightblue'))
    for i in range(num_simulator_parameters.value):
        sp_def = Text(value=f"Sim_Parameter_{i}",
                      placeholder=f"Sim_Parameter_{i}",
                      description=f"{i+1}-th Parameter of Simulator:",
                      disabled=False)
        
        sp_val = Text(value="val",
                      placeholder="val",
                      description="Value:",
                      disabled=False)

        sp_numeric =  Checkbox(value=False,
                           description='Numeric',
                           disabled=False,
                           indent=False)
        
        num_simulator_parameters_names.append(sp_def)
        num_simulator_parameters_values.append(sp_val)
        num_simulator_parameters_children.extend([sp_def, sp_val, sp_numeric])


    grid = layout_generator(num_simulator_parameters_header, num_simulator_parameters_children, num_cols=3, col_width=400)
    display(grid)

GridspecLayout(children=(Button(description='Simulator Parameters', layout=Layout(grid_area='widget001', width…

An example yaml config

```yaml
reward:
  step_reward:
    P_s:
      expr: P_s
      coef: 1.0
  terminal_reward:
    P_s:
      expr: P_s
      coef: 1.0
  input_reward:
    inp: 1.0
```
---- 

In [49]:
# define reward function

sv_names = [val.value for val in state_variables_names]
sv_names
in_names = [val.value for val in control_variables_names]
in_names

if len(sv_names) > 0:
    state_reward_header = Button(description='Define State Reward',
                                layout=Layout(width='auto', grid_area='header'),
                                style=ButtonStyle(button_color='lightblue'))
    step_rewards_expr = []
    step_rewards_coef = []
    terminal_rewards_expr = []
    terminal_rewards_coef = []
    state_rewards_children = []
    for i in range(len(sv_names)):
        sr_def = Label(value=sv_names[i],
                       disabled=False)

        sr_expr = Text(value=f"({sv_names[i]}-0.6)",
                            placeholder=f"({sv_names[i]}-0.6)",
                            description="Step Expr:",
                            disabled=False)

        sr_coef = FloatText(value=0.0,
                            description=f"Step Coef:",
                            disabled=False)
        
        tr_expr = Text(value=f"({sv_names[i]}-0.6)",
                            placeholder=f"({sv_names[i]}-0.6)",
                            description="Terminal Expr:",
                            disabled=False)

        tr_coef = FloatText(value=0.0,
                            description=f"Terminal Coef:",
                            disabled=False)
        
        step_rewards_expr.append(sr_expr)
        step_rewards_coef.append(sr_coef)
        terminal_rewards_expr.append(tr_expr)
        terminal_rewards_coef.append(tr_coef)
        state_rewards_children.extend([sr_def, sr_expr, sr_coef, tr_expr, tr_coef])

    grid = layout_generator(state_reward_header, state_rewards_children, num_cols=5, col_width=250)
    display(grid)
            

GridspecLayout(children=(Button(description='Define State Reward', layout=Layout(grid_area='widget001', width=…

In [50]:
if len(in_names) > 0:
    input_reward_header = Button(description='Define Input Reward',
                                layout=Layout(width='auto', grid_area='header'),
                                style=ButtonStyle(button_color='lightblue'))
    input_rewards_coef = []
    input_rewards_children = []
    for i in range(len(in_names)):
        in_def = Label(value=in_names[i], disabled=False)

        in_coef = FloatText(value=0.0,
                            description=f"Coef:",
                            disabled=False)
        input_rewards_coef.append(in_coef)
        input_rewards_children.extend([in_def, in_coef])

    grid = layout_generator(input_reward_header, input_rewards_children, num_cols=2, col_width=400)
    display(grid)

GridspecLayout(children=(Button(description='Define Input Reward', layout=Layout(grid_area='widget001', width=…

An example yaml config

```yaml
mpc:
  setup:
    n_horizon: 20
    n_robust: 0
    open_loop: 0
    t_step: 1.0
    state_discretization: collocation
    collocation_type: radau
    collocation_deg: 2
    collocation_ni: 2
    store_full_solution: True
  bounds:
    X_s:
      lower: 0.0 # required
      upper: 3.7 # required
      soft: False # default, can be omitted
    S_s:
      lower: -0.01
      upper: 2.0
    P_s:
      lower: 0.0
      upper: 3.0
    V_s:
      lower: 0.0
      upper: 150.0
    inp:
      lower: 0.0
      upper: 0.2
  uncertainities:
    Y_x: [0.5, 0.4, 0.3]
    S_in: [200.0, 220.0, 180.0]

```
---- 

An example yaml config

```yaml
estimator:
  type: StateFeedback # StateFeedback, EKF(Extended Kalman Filter), ..
  parameters: null
```
---- 